# Teste A/B para comparar duas versões de um site de e-commerce

Uma empresa de e-commerce quer testar se um novo design em seu site aumenta o número de conversões (clientes que entram na página e compram) em 21%. Após verificarmos que o aumento de conversão é de fato possível via mudança no design, decidimos realizar um experimento (há espaço no mercado para aumento, e fatores relacionados à qualidade do site, como tempo de carregamento, não estão influenciando uma baixa conversão).

Portanto, esse projeto compara duas versões de página de site de e-commerce em relação à conversão por meio de um teste A/B.

Durante o período de teste, um determinado número de usuários foi direcionado para a versão atual da página (grupo de controle) e outro grupo viu a nova versão (grupo de tratamento). Ao final do experimento, os resultados dos grupos foram comparados para determinar se a diferença de conversão entre eles era estatísticamente significativa.

## Importando bibliotecas

In [1]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

## Etapa 1: Escolha da métrica de sucesso e medida do seu valor atual

### Definição das hipóteses do teste

Definimos as hipóteses do nosso teste da seguinte forma:

Hₒ: p = pₒ

Hₐ: p ≠ pₒ

Sendo:

p -> Taxa de conversão do site novo

pₒ -> Taxa de conversão do site antigo

Nosso nível de confiança será de 95% (alpha = 5%).

O teste é bicaudal porque não sabemos se o desempenho da nova página será melhor, pior ou igual que a página atual. 
Não há problema em realizar o teste bicaudal porque se o resultado for significativo e o valor de p for maior que pₒ na amostra, também o será na população.

### Cálculo da medida atual da métrica de sucesso

A métrica de sucesso a ser analisada é a taxa conversão do cliente, ou seja, se ele efetuou a compra após ver a página do site.

Para calcular seu valor atual, vamos usar resultados de conversão de usuários que foram previamente registrados no site atual. 

In [4]:
df_metrica_atual = pd.read_csv('../data/df_anterior.csv')
df_metrica_atual.head()

,Unnamed: 0,user_id,timestamp,converted
0,4402,759903,2017-01-21 20:33:04.550046,0
1,214023,889087,2017-01-16 16:59:09.713050,0
2,143767,896579,2017-01-10 03:04:35.928212,0
3,192434,748209,2017-01-23 22:31:58.061727,0
4,139934,935283,2017-01-07 18:17:28.683289,0


Na coluna "converted":

0 -> O usuário não realizou compra

1 -> O usuário realizou compra

In [7]:
# Quantidade de usuários que converteu
converteu = df_metrica_atual[df_metrica_atual['converted'] == 1].converted.sum()

# Quantidade total de ususários
total_usuários = df_metrica_atual.user_id.count()

# Taxa de conversão
taxa_conversao_atual = round(converteu/total_usuários, 3)

print(f'A taxa de conversão atual é {taxa_conversao_atual}.')

A taxa de conversão atual é 0.122.


### Cálculo da taxa de conversão que o negócio deseja atingir com a nova página do site

A meta do negócio (baseada em pesquisas de mercado e estudos similares) é aumentar a taxa de conversão em 21%. Logo, precisamos calcular qual seria o valor da taxa de conversão esperada.

Ao estabelecer uma meta é preciso levar em conta que o aumento desejado deve ser significativo do ponto de vista prático. No caso que estamos trabalhando, o aumento de conversão deve justificar financeiramente o investimento de alterar a página do site.

In [9]:
# Taxa de conversão que se deseja atingir
meta = round(taxa_conversao_atual * 1.21, 3)

print(f'A taxa de conversão desejada é {meta}.')

A taxa de conversão desejada é 0.148.


## Etapa 2: Determinação de como será a amostra

Precisamos que tanto o grupo de controle, quanto o grupo de tratamento sejam escolhidos de forma aleatória e que representem bem a população. Além disso, precisamos garantir que os usuários escolhidos em cada grupo sejam semelhantes, ou seja, as proporções de idade, gênero e perfil de compra sejam semelhantes em ambos os grupos.

Precisamos, também, garantir que as amostras sejam coletadas durante o mesmo período de tempo para evitar que a influência de fatores como sazonalidade ou promoções afetem um dos grupos.

## Etapa 3: Cálculo do tamanho mínimo da amostra

Um tamanho maior de amostra nos permite estimativas mais precisas. No entanto, uma amostra muito grande acarreta em custos mais altos. Portanto, precisamos determinar uma amostra mínima (suficiente) para comprovar um efeito caso ele exista.

O cálculo do tamanho da amostra está relacionado à análise de poder do teste. O poder estatístico é a probabilidade de se detectar corretamente um efeito; o poder é influenciado por três fatores:

- Tamanho da amostra: tamanhos maiores permitem a detecção de efeitos menores pelo teste de hipóteses.
- Tamanho do efeito: quanto maior o tamanho do efeito, menor a probabilidade de ser um erro aleatório.
- Variabilidade: Quando os dados da amostra têm maior variabilidade, é mais provável que o erro de amostragem aleatória produza diferenças consideráveis entre os grupos experimentais, mesmo quando não há efeito real.

Portanto, para calcular o tamanho amostral necessário para um experimento, precisamos especificar o poder estatísico, o tamanho do efeito que queremos atingir e a variabilidade.

### Uso do G*Power para calcular o tamanho da amostra

Vamos usar o software G*Power para calcular o tamanho da amostra.

O teste que queremos realizar é um teste Z para diferença de proporções entre duas amostras independentes.

Passamos as seguintes variáveis de entrada:

- Tails: Two (teste bicaudal)
- Proportions p2: 0.148 (meta)
- Proportions p1: 0.122 (atual)
- alpha: 0.05
- Power: 80% (referência padrão em estudos)
- Alocation ratio: 1 (queremos que ambos os grupos tenham tamanho igual)

![image.png](../images/gpower.png)

Precisamos de uma amostra com tamanho de 5422 (2711 sessões de usuários do site para cada grupo).

## Etapa 4: Realização do teste e comparação dos resultados

Após realizar a coleta de dados conforme planejado nas etapas anteriores, temos um dataset com os resultados dos usuários dos grupos de controle e tratamento (se converteram ou não).

0 -> Não converteu

1 -> Converteu

In [3]:
df_teste = pd.read_csv('../data/df_test.csv')
df_teste.head()

,Unnamed: 0,user_id,timestamp,landing_page,converted,group
0,281655,918193,2017-01-03 13:42:27.285433,new_page,1,treatment
1,7271,679658,2017-01-20 02:00:54.448676,new_page,1,treatment
2,269760,815158,2017-01-16 05:11:33.823264,new_page,1,treatment
3,158882,857307,2017-01-22 08:58:50.687626,new_page,1,treatment
4,153958,716896,2017-01-15 17:59:03.065352,new_page,1,treatment


In [10]:
# Resultados: taxa de conversão de cada grupo

resultado_tratamento = round(df_teste[df_teste['group'] == 'treatment'].converted.mean(), 3)

resultado_controle = round(df_teste[df_teste['group'] == 'control'].converted.mean(), 3)

print(f'A taxa do grupo tratamento foi de: {resultado_tratamento}.\nA taxa do grupo controle foi de: {resultado_controle}.')

A taxa do grupo tratamento foi de: 0.19.
A taxa do grupo controle foi de: 0.125.


Esse é o resultado na amostra. Precisamos agora realizar o teste de hipóteses para comparação de duas proporções para verificar se esse comportamento também é válido para a população, ou seja, se a diferença que observamos entre os grupos é estatísticamente significativa.

### Teste Z para comparação de duas proporções

In [11]:
# Calculando a quantidade de sucessos que tivemos em cada grupo (ou seja, quantas conversões tivemos)

sucessos_controle = df_teste[df_teste['group'] == 'control']['converted'].sum()

sucessos_tratamento = df_teste[df_teste['group'] == 'treatment']['converted'].sum()

sucessos = [sucessos_controle, sucessos_tratamento]

# Calculando o tamanho de cada grupo

qtd_controle = df_teste[df_teste['group'] == 'control']['user_id'].count()

qtd_tratamento = df_teste[df_teste['group'] == 'treatment']['user_id'].count()

nobs = [qtd_controle, qtd_tratamento]

z_test, p_value = proportions_ztest(sucessos, nobs=nobs)

print(f'A estatística de teste Z é: {z_test}.\nO p-valor é: {p_value}.')

A estatística de teste Z é: -6.733481162342496.
O p-valor é: 1.6565111652777357e-11.


O p-valor obtido é menor que o nível de significância estabelecido, logo, há evidências estatísticas para rejeitarmos a hipótese nula em favor da hipótese alternativa. Em outras palavras, a diferença entre as proporções do grupo de controle e do grupo de tratamento é estatísicamente significativa.

Podemos também rejeitar a hipótese nula pelo valor da estatística Z. Para o teste bilateral com alpha = 5%, a zona de rejeição é Z < -1.96 e z > 1.96.

Com isso, concluímos que a taxa de conversão da nova página é maior que a da página atual e recomendamos a implementação definitiva da página nova.

### Cálculo do intevalo de confiança para as proporções

Também podemos chegar ao mesmo resultado do teste de hipóteses (rejeição da hipótese nula) pelo cálculo do intervalo de confiança.

Além disso, o intervalo de confiança nos dá uma estimativa da margem de erro do valor que obtivemos para a taxa de conversão da pagina nova.

In [12]:
(lower_control, lower_treatment), (upper_control, upper_treatment) = proportion_confint(sucessos, nobs=nobs, alpha=0.05)

print(f'O intervalo de confiança a 95% para o grupo controle é: [{lower_control:.3f}, {upper_control:.3f}].')
print(f'O intervalo de confiança a 95% para o grupo tratamento é: [{lower_treatment:.3f}, {upper_treatment:.3f}].')

O intervalo de confiança a 95% para o grupo controle é: [0.112, 0.137].
O intervalo de confiança a 95% para o grupo tratamento é: [0.175, 0.204].


Os intervalos de confiança não se sobrepõem. Portanto, podemos concluir que as proporções são diferentes.

## Conclusão

O experimento conduzido comprovou estatísticamente que a alteração na página melhora a conversão dos usuários.

Com a mudança definitiva para a nova página, podemos esperar uma nova taxa de conversão variando entre 0.175 e 0.204.